In [ ]:
###### import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch
import h5py
import os
import sys
import scipy
import damselfly as df
import scipy.signal

PATH = '/storage/home/adz6/group/project'
RESULTPATH = os.path.join(PATH, 'results/damselfly')
PLOTPATH = os.path.join(PATH, 'plots/damselfly')
DATAPATH = os.path.join(PATH, 'damselfly/data/datasets')
SIMDATAPATH = os.path.join(PATH, 'damselfly/data/sim_data')

"""
Date: 6/25/2021
Description: template
"""

def GetTrackLength(rng, slicesize, mean=3, signal_length=8192 ):
    est_track_length = int(slicesize * rng.exponential(mean))
    if est_track_length >= signal_length:
        return signal_length
    else:
        return est_track_length
    
def noise(rng, var, size, shape):
    noise = rng.multivariate_normal([0,0], np.eye(2) * var / 2, size)
    noise = noise[:, 0] + 1j * noise[:, 1]
    
    return noise.reshape(shape)

dataset_file = '210730_84_1d2sl_eval.h5'
h5datafile = h5py.File(os.path.join(SIMDATAPATH, dataset_file), 'r')

var = 1.38e-23 * 10 * 50 * 200e6

signals = h5datafile['data'][:]



noise_samples = 1
#shape = signals.shape
#size = signals.size

track_lengths = np.arange(256, 2*8192 + 256, 256)
#track_lengths = np.arange()
ntrack = track_lengths.size

nsignals = signals.shape[0]

noise_scores = np.zeros((ntrack, noise_samples, nsignals, nsignals))
signal_scores = np.zeros((ntrack, noise_samples, nsignals, nsignals))

for i, track_len in enumerate(track_lengths):
    for n in range(noise_samples):
        print(track_len)
        aug_signals = np.zeros((signals.shape[0], track_len), dtype=np.complex64) 
            
        aug_signals = signals[:, 0:track_len]
        
        norm = 1 / np.sqrt(var * np.diag(np.matmul(aug_signals, aug_signals.conjugate().T)))
        
        #print(norm.shape, aug_signals.shape, templates.shape)
        
        templates = aug_signals * norm.reshape((norm.size, 1)).repeat(track_len, axis=-1)
        
        noisy_data = aug_signals + noise(rng, var, aug_signals.size, aug_signals.shape)

        #x = abs(scipy.signal.fftconvolve(noisy_data, templates, axes=-1, mode='valid'))
        x = abs(np.matmul(noisy_data, templates.conjugate().T))
        signal_scores[i, n, :] = x

    
        #x = abs(np.matmul(noise(rng, var, templates.size, templates.shape), templates.conjugate().T))
        #x = abs(np.matmul(noise(rng, var, size, shape)[:,0:template_size], templates.conjugate().T))
        #print(np.max(x, axis=-1), np.mean(x))
        #print(x)
        #noise_scores[i, n, :] = x
        
        #if i % 5 == 4:
        #    print(f'{i + 1} / {ntrack * noise_samples}')

result_name = '210731_matched_filter_scores_variable_track_length_10K.npz'

np.savez(os.path.join(RESULTPATH, result_name), signal=signal_scores, noise=noise_scores, track_lengths=track_lengths, noise_samp=noise_samples)


In [ ]:
signal = signals[3489, 0:]

n = noise(rng, var, 16384, 16384)
print(signal.shape)
template = signal * 1 / np.sqrt(var * np.vdot(signal, signal))

#print(abs(np.vdot(signal[0:8192], template[0:8192])))

print(abs(np.vdot(signal[0:], template[0:])))

In [ ]:
plt.plot(signals[0])

In [ ]:
signal = signals[3489, 0:4096]
print(signal.shape)

n = noise(rng, var, 8192, 8192)
template = signal * 1 / np.sqrt(var * np.vdot(signal, signal))

#print(abs(np.vdot(signal[0:4096], template[0:4096])))

print(abs(np.vdot(signal[0:], template[0:])))

In [ ]:
rng = np.random.default_rng()

In [ ]:
list1 = []
for i in range(10000):
    n_real = rng.normal(0, np.sqrt(var / 2), 8192)
    h_n = np.dot(n_real, template.real)
    list1.append(h_n)
print(np.sqrt(template ** 2))
print(np.std(list1))

In [ ]:
plt.plot(template.imag)

In [ ]:
metadata = {}
for n, dset in enumerate(h5datafile['meta']):
    metadata[dset] = h5datafile['meta'][dset][:]
    
metadata = pd.DataFrame(metadata)



In [ ]:
template_size = 8192

norm = 1 / np.sqrt(var * np.diag(np.matmul(signals[:, 0:template_size], signals[:, 0:template_size].conjugate().T)))

templates = signals[:, 0:template_size] * norm.reshape((norm.size, 1)).repeat(template_size, axis=-1)

In [ ]:
print(abs(np.vdot(templates[0, :], signals[0, 0:template_size])))
print(np.max(abs(scipy.signal.fftconvolve(templates[0, :], signals[0, :], mode='valid'))))

track_len = 4096
aug_signal = np.zeros(signals[0, :].shape, dtype=np.complex64) 

aug_signal[0:track_len] = signals[0, 0:track_len]

print(np.max(abs(scipy.signal.fftconvolve(templates[0, :], aug_signal, mode='valid'))))

In [ ]:
rng = np.random.default_rng()
slicesize = 8192
ntrack = 1000
mean_length = 3
track_lengths = np.zeros(ntrack, dtype=np.int32)

for n in range(ntrack):
    track_lengths[n] = GetTrackLength(rng, slicesize, mean=mean_length, signal_length=signals.shape[-1])

In [ ]:
plt.hist(track_lengths, 32)

In [ ]:
noise_samples = 5
#shape = signals.shape
#size = signals.size

track_lengths = np.arange(512, 2*8192 + 512, 512)
ntrack = track_lengths.size

nsignals = signals.shape[0]

noise_scores = np.zeros((ntrack * noise_samples, nsignals, nsignals))
signal_scores = np.zeros((ntrack * noise_samples, nsignals, nsignals))

for i, track_len in enumerate(track_lengths):
    for n in range(noise_samples):
        print(track_len)
        aug_signals = np.zeros((signals.shape[0], track_len), dtype=np.complex64) 
            
        aug_signals = signals[:, 0:track_len]
        
        norm = 1 / np.sqrt(var * np.diag(np.matmul(aug_signals, aug_signals.conjugate().T)))
        
        #print(norm.shape, aug_signals.shape, templates.shape)
        
        templates = aug_signals * norm.reshape((norm.size, 1)).repeat(track_len, axis=-1)
        
        noisy_data = aug_signals + noise(rng, var, aug_signals.size, aug_signals.shape)

        #x = abs(scipy.signal.fftconvolve(noisy_data, templates, axes=-1, mode='valid'))
        x = abs(np.matmul(noisy_data, templates.conjugate().T))
        print(x)
        signal_scores[i, :] = x

    
        x = abs(np.matmul(noise(rng, var, templates.size, templates.shape), templates.conjugate().T))
        #x = abs(np.matmul(noise(rng, var, size, shape)[:,0:template_size], templates.conjugate().T))
        #print(np.max(x, axis=-1), np.mean(x))
        print(x)
        noise_scores[i, :] = x
        
        if i % 10 == 9:
            print(f'{i + 1} / {ntrack}')

result_name = '210731_matched_filter_scores_mt3_10K.npz'

np.savez(os.path.join(RESULTPATH, result_name), signal=signal_scores, noise=noise_scores, track_len=track_lengths)
        

In [ ]:
print(track_lengths)